In [1]:
import warnings
warnings.simplefilter("ignore")

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [3]:
from datasets import load_dataset

from opacus.validators import ModuleValidator
from opacus.utils.batch_memory_manager import BatchMemoryManager
from opacus import PrivacyEngine

import torch
import torch.nn as nn
import numpy as np

from tqdm.notebook import tqdm
from torch.optim import SGD
from torch.utils.data import DataLoader

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, DataCollatorWithPadding, Trainer, TrainingArguments

from sklearn.metrics import accuracy_score

In [4]:
model_name = "prajjwal1/bert-tiny"
EPOCHS = 4
BATCH_SIZE = 32
LR = 2e-5

In [5]:
# Prepare data
dataset = load_dataset("glue", "sst2")
num_labels = dataset["train"].features["label"].num_classes

train_dataset = dataset["train"]
val_dataset = dataset["validation"]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
def preprocess_function(examples):
    tokenized_dataset = tokenizer(examples["sentence"], truncation=True, padding='max_length', max_length=128)
    return tokenized_dataset

In [8]:
tokenized_train_data = train_dataset.map(preprocess_function, batched=True)
tokenized_train_data.remove_columns(['sentence', 'idx'])
tokenized_train_data.rename_column('label', 'labels')
tokenized_train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [9]:
tokenized_val_data = val_dataset.map(preprocess_function, batched=True)
tokenized_val_data.remove_columns(['sentence', 'idx'])
tokenized_val_data.rename_column('label', 'labels')
tokenized_val_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
EPSILON = 8.0
DELTA = 1/len(tokenized_train_data)
MAX_GRAD_NORM = 0.01

In [13]:
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 2

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.train().to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
optimizer = SGD(model.parameters(), lr=LR)

In [18]:
dataloader = DataLoader(tokenized_train_data, batch_size=BATCH_SIZE, collate_fn=data_collator)

In [20]:
privacy_engine = PrivacyEngine(accountant="rdp")

model, optimizer, tokenized_train_data = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=dataloader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
    batch_first=True,    
)

In [21]:
print(f"Using Sigma = {optimizer.noise_multiplier:.3f} | C = {optimizer.max_grad_norm} | Initial DP (ε, δ) = ({privacy_engine.get_epsilon(DELTA)}, {DELTA})")

Using Sigma = 0.417 | C = 0.01 | Initial DP (ε, δ) = (0, 1.4848030408766277e-05)


In [22]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable Parameters: {trainable_params} || All Parameters: {all_param} || Trainable Parameters (%): {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)

Trainable Parameters: 4386178 || All Parameters: 4386178 || Trainable Parameters (%): 100.00


In [ ]:
training_args = TrainingArguments(
    output_dir="./output/xlnet_0",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()